In [1]:
import pandas as pd
file_path = r"C:\Users\Sivaraman\Desktop\study\data analytics\Data (for practice)\Bank_details_new1.xlsx"
df = pd.read_excel(file_path)
print(df)

     CustomerId        Name  CreditScore  Country  Gender  Age  Tenure  \
0      15634602    Hargrave          619   France  Female   42       2   
1      15634602    Hargrave          619   France  Female   42       2   
2      15634602        Onio          502   France  Female   42       8   
3      15634602        Onio          502   France  Female   42       8   
4      15737888    Mitchell          850    Spain  Female   43       2   
..          ...         ...          ...      ...     ...  ...     ...   
448    15694860    Uspensky          675   France  Female   38       6   
449    15658169        Cook          778    Spain  Female   47       6   
450    15794396     Newbold          494  Germany  Female   38       7   
451    15785798  Uchechukwu          850   France    Male   40       9   
452    15710825       Ch'en          592    Spain    Male   31       7   

       Balance  NumOfProducts     Salary       Date      Dep  Cumulative_Sal  \
0         0.00              1  

In [4]:
#transform() - Same shape transformation (vectorized)
#Add group mean
df['dep_avg_sal'] = df.groupby('Dep')['Salary'].transform('mean')
print(df[['Salary', 'dep_avg_sal']])




        Salary    dep_avg_sal
0    101348.88   81025.139412
1    101348.88   81025.139412
2    112542.58   81025.139412
3    112542.58   81025.139412
4     79084.10   81025.139412
..         ...            ...
448  138777.00  105477.168519
449  124694.99  105477.168519
450   40084.32  105477.168519
451  119232.33  105477.168519
452   43921.36  105477.168519

[453 rows x 2 columns]


In [5]:
#You're normalizing the salaries within each department using Z-score normalization.

#uses
'''
✅ Why is this useful?
Some departments might have higher or lower salary ranges.

Normalizing helps when comparing data across groups.

Especially useful in machine learning and statistical analysis.
'''

df['Norm_sal'] = df.groupby('Dep')['Salary'].transform(lambda x : (x-x.mean())/ x.std() )
print(df[['Salary', 'Norm_sal']])

        Salary  Norm_sal
0    101348.88  0.567726
1    101348.88  0.567726
2    112542.58  0.880413
3    112542.58  0.880413
4     79084.10 -0.054221
..         ...       ...
448  138777.00  0.747110
449  124694.99  0.431168
450   40084.32 -1.467144
451  119232.33  0.308609
452   43921.36 -1.381057

[453 rows x 2 columns]


In [7]:
# Find max in each group
df['Max_sal'] = df.groupby('Dep')['Salary'].transform('max')
print(df[['Dep', 'Max_sal']])

         Dep    Max_sal
0       Bank  149756.71
1       Bank  149756.71
2       Bank  149756.71
3       Bank  149756.71
4       Bank  149756.71
..       ...        ...
448  Entrepr  199645.45
449  Entrepr  199645.45
450  Entrepr  199645.45
451  Entrepr  199645.45
452  Entrepr  199645.45

[453 rows x 2 columns]


In [14]:
# apply() - Custom group logic (can change structure)
# 1. Get top salary per department
top_sal = df.groupby('Dep').apply(lambda x : x.nlargest(1, 'Salary'), include_groups = False).reset_index(drop = True)
print(top_sal)

# include_groups = False, its is used to prevent the warning

'''By default, the grouping column 'Dep' is included in each group x.

But in newer versions (and future ones), pandas plans to exclude the grouping column from the group x by default.'''


   CustomerId        Name  CreditScore  Country  Gender  Age  Tenure  \
0    15574012         Chu          645    Spain    Male   44       8   
1    15599792     Dimauro          545   France  Female   26       1   
2    15676715   Madukaego          640   France    Male   68       9   
3    15625047         Yen          846   France  Female   38       5   
4    15661670    Chidozie          524  Germany  Female   31       8   
5    15743040  Kuznetsova          724  Germany    Male   41       2   
6    15577657    McDonald          732   France    Male   41       8   
7    15755196      Lavine          834   France  Female   49       2   

     Balance  NumOfProducts     Salary       Date  Cumulative_Sal  \
0  113755.78              2  149756.71 2023-01-06       656623.73   
1       0.00              2  199638.56 2024-02-25      2767624.70   
2       0.00              2  199493.38 2024-06-13      7627290.64   
3       0.00              1  187616.16 2023-02-23       187616.16   
4  107

In [20]:
#  Custom aggregation with multiple columns

def custom_summ(x):
    return pd.Series({
        'Max_Sal' : x['Salary'].max(),
        'Min_Sal' : x['Salary'].min(),
        'Range' : x['Salary'].max() - x['Salary'].min()
    })
summary = df.groupby('Dep').apply(custom_summ, include_groups = False).reset_index() # include_groups = False it is used to prevent the old and new version error.
print(summary)

# sometime one kind of warning is came.
#what is tells you

'''🟡 What is DeprecationWarning?
It’s a warning that tells you:

⚠️ “This thing you’re using still works now, but it might stop working in future versions of Python or Pandas.'''

# fix it: include_groups = False

        Dep    Max_Sal   Min_Sal      Range
0      Bank  149756.71   5097.67  144659.04
1  Business  199638.56   4861.72  194776.84
2       Clg  199493.38    371.05  199122.33
3        DA  187616.16  38433.35  149182.81
4   Doctors  199725.39    600.36  199125.03
5   Entrepr  199645.45  27474.81  172170.64
6        IT  170886.17   8487.75  162398.42
7    Police  194365.76  27822.99  166542.77


In [28]:
# Add Rank within group


df['Salary_rank'] = df.groupby('Dep')['Salary'].transform(lambda x: x.rank(ascending = False))
print(df[['Salary', 'Salary_rank']])

# .apply() might break if your DataFrame's row order/index is changed. .transform() is always safe ✅

        Salary  Salary_rank
0    101348.88          5.5
1    101348.88          5.5
2    112542.58          3.5
3    112542.58          3.5
4     79084.10          8.0
..         ...          ...
448  138777.00          6.0
449  124694.99         10.0
450   40084.32         26.0
451  119232.33         13.0
452   43921.36         24.0

[453 rows x 2 columns]
